In [1]:
import pandas as pd
import numpy as np
import altair as alt
from io import StringIO

# 1. Carga de Datos (Usando los datos simulados previamente)
# [Se mantiene la misma estructura de datos, ya que la diferencia es solo conceptual (metros vs metros/peso)]
data = """
Categoria;Prueba;Marca;Año;Competencia;Nombre Atleta
Escolar Masculino;800m;02:02:17;2023;Nacional Escolar;Valentín Acuña
Escolar Masculino;800m;02:04:02;2023;Nacional Escolar;Tomas Leiva
Escolar Masculino;800m;02:09:30;2023;Nacional Escolar;Nicolas Orellana
Escolar Masculino;800m;02:09:35;2023;Nacional Escolar;Alonso Acuña
Escolar Masculino;800m;02:12:09;2023;Nacional Escolar;Daniel Rispatron
Escolar Masculino;800m;02:14:27;2023;Nacional Escolar;Matias Armengolli
Escolar Masculino;Salto Largo;5.89;2023;Nacional Escolar;Agustin Dominguez
Escolar Masculino;Salto Largo;5.82;2023;Nacional Escolar;Carlos Vergara
Escolar Masculino;Lanzamiento Bala;16.51;2023;Nacional Escolar;Ricardo Castillo
Escolar Masculino;Lanzamiento Bala;15.35;2023;Nacional Escolar;Bastián Leiva
Escolar Femenino;800m;02:22:15;2023;Nacional Escolar;Catalina Rojas
Escolar Femenino;800m;02:25:05;2023;Nacional Escolar;Fernanda Soto
Escolar Femenino;Salto Largo;4.89;2023;Nacional Escolar;Josefa Flores
Escolar Femenino;Salto Largo;4.82;2023;Nacional Escolar;Martina López
Escolar Femenino;Lanzamiento Bala;11.51;2023;Nacional Escolar;Daniela Fuentes
Escolar Femenino;Lanzamiento Bala;10.35;2023;Nacional Escolar;Florencia Vega
Adulto Masculino;Lanzamiento Bala;17.59;2024;Nacional Adulto;Ignacio Morales
Adulto Masculino;Lanzamiento Bala;17.3;2024;Nacional Adulto;Felipe Cárdenas
Adulto Masculino;Lanzamiento Disco;50.15;2024;Nacional Adulto;Ignacio Morales
Adulto Masculino;Lanzamiento Disco;48.5;2024;Nacional Adulto;Felipe Cárdenas
Adulto Masculino;100m;10.55;2024;Nacional Universitario;Juan Soto
Adulto Masculino;100m;10.65;2024;Nacional Universitario;Pedro Alarcón
Adulto Femenino;Lanzamiento Bala;15.49;2025;Nacional Adulto;Mariela Pérez
Adulto Femenino;Lanzamiento Bala;12.78;2025;Nacional Adulto;Fernanda Venegas
Adulto Femenino;Lanzamiento Disco;55.15;2025;Nacional Adulto;Karen Gallardo
Adulto Femenino;Lanzamiento Disco;41.77;2025;Nacional Adulto;Javiera Bravo
Adulto Femenino;100m;12.5;2025;Nacional Adulto;María José Soto
Adulto Femenino;100m;12.6;2025;Nacional Adulto;Fernanda Venegas
Escolar Masculino;800m;02:02:10;2024;Nacional Escolar;Valentín Acuña
Escolar Masculino;800m;02:03:50;2024;Nacional Escolar;Tomas Leiva
Escolar Masculino;Salto Largo;5.95;2024;Nacional Escolar;Agustin Dominguez
Escolar Masculino;Lanzamiento Bala;16.55;2024;Nacional Escolar;Ricardo Castillo
Adulto Masculino;Lanzamiento Bala;17.65;2025;Nacional Adulto;Ignacio Morales
Adulto Masculino;Lanzamiento Disco;50.25;2025;Nacional Adulto;Ignacio Morales
Adulto Masculino;100m;10.85;2025;Nacional Adulto;Juan Soto
Adulto Masculino;100m;10.95;2025;Nacional Adulto;Pedro Alarcón
Adulto Femenino;Lanzamiento Bala;15.55;2025;Nacional Adulto;Mariela Pérez
Adulto Femenino;Lanzamiento Disco;55.25;2025;Nacional Adulto;Karen Gallardo
Escolar Femenino;800m;02:22:05;2025;Nacional Escolar;Catalina Rojas
Escolar Femenino;Salto Largo;4.95;2025;Nacional Escolar;Josefa Flores
Escolar Masculino;800m;02:02:05;2025;Nacional Escolar;Valentín Acuña
Escolar Masculino;Salto Largo;6.05;2025;Nacional Escolar;Agustin Dominguez
Escolar Masculino;Lanzamiento Bala;16.65;2025;Nacional Escolar;Ricardo Castillo
Adulto Masculino;Lanzamiento Bala;17.75;2023;Nacional Adulto;Ignacio Morales
Adulto Masculino;Lanzamiento Disco;50.05;2023;Nacional Adulto;Ignacio Morales
Adulto Masculino;100m;11.05;2023;Nacional Adulto;Juan Soto
Adulto Femenino;Lanzamiento Bala;15.35;2023;Nacional Adulto;Mariela Pérez
Adulto Femenino;Lanzamiento Disco;55.05;2023;Nacional Adulto;Karen Gallardo
Adulto Femenino;100m;12.95;2023;Nacional Adulto;María José Soto
"""
df = pd.read_csv(StringIO(data), delimiter=";")

# 2. Conversión de Marca
def convertir_marca_a_float(marca):
    """Convierte marcas de tiempo (MM:SS:cc) a segundos o deja la marca numérica como float (metros)."""
    if isinstance(marca, str) and ':' in marca:
        try:
            parts = marca.split(':')
            if len(parts) == 3:
                minutes = int(parts[0])
                seconds = int(parts[1])
                hundredths = int(parts[2])
                return minutes * 60 + seconds + hundredths / 100
            elif len(parts) == 2:
                minutes = int(parts[0])
                seconds = float(parts[1].replace(',', '.'))
                return minutes * 60 + seconds
            else:
                return np.nan
        except:
            return np.nan
    else:
        try:
            return float(str(marca).replace(',', '.'))
        except:
            return np.nan

df['Marca_Num'] = df['Marca'].apply(convertir_marca_a_float)
df.dropna(subset=['Marca_Num'], inplace=True)

# 3. Clasificación de Prueba - AJUSTE DE NOMENCLATURA
# Las pruebas que tienen ':' son TIEMPO. Las que no, son Distancia/Metros.
df['Tipo_Prueba'] = np.where(df['Marca'].str.contains(':').fillna(False),
                            'Tiempo (Menor es Mejor)',
                            'Distancia/Metros (Mayor es Mejor)')
df['Categoria'] = df['Categoria'].apply(lambda x: x.strip())
df['Competencia'] = df['Competencia'].apply(lambda x: x.strip())

# 4. Filtrar solo competencias Nacionales
competencias_nacionales = ['Nacional Escolar', 'Nacional Adulto']
df_nacional = df[df['Competencia'].isin(competencias_nacionales)]

# 5. Cálculo de la media del rendimiento por Año, Tipo_Prueba y Categoria
df_media_anual = df_nacional.groupby(['Año', 'Tipo_Prueba', 'Categoria'], as_index=False)['Marca_Num'].mean()
df_media_anual.rename(columns={'Marca_Num': 'Marca_Num_Media'}, inplace=True)

# 6. Visualización con Altair

# Orden para las categorías y colores
orden_categorias = ['Adulto Masculino', 'Adulto Femenino', 'Escolar Masculino', 'Escolar Femenino']
color_scale = alt.Scale(domain=orden_categorias, range=['#1f77b4', '#ff7f0e', '#aec7e8', '#ffbb78'])

# Creación del gráfico de líneas base
base = alt.Chart(df_media_anual).encode(
    x=alt.X('Año:O', title='Año', axis=alt.Axis(labelAngle=0)), # O: Ordinal para años
    color=alt.Color('Categoria', scale=color_scale, legend=alt.Legend(title="Categoría de Atleta")),
    tooltip=['Año', 'Categoria', alt.Tooltip('Marca_Num_Media', title='Media de Marca', format='.3f')]
)

# -----------------
# Gráfico de Distancia/Metros (Mayor es Mejor) - Eje Y no invertido
# -----------------
distancia_chart = base.transform_filter(
    alt.FieldEqualPredicate(field='Tipo_Prueba', equal='Distancia/Metros (Mayor es Mejor)')
).encode(
    # AJUSTE DE ETIQUETA
    y=alt.Y('Marca_Num_Media', title='Media de Marca (Metros) - Mayor es Mejor'),
).mark_line(point=True).properties(
    # AJUSTE DE TÍTULO
    title='Distancia/Metros (Lanzamientos/Saltos) - Mejora = Línea Ascendente'
)

# -----------------
# Gráfico de Tiempo (Menor es Mejor) - Eje Y invertido
# -----------------
tiempo_chart = base.transform_filter(
    alt.FieldEqualPredicate(field='Tipo_Prueba', equal='Tiempo (Menor es Mejor)')
).encode(
    # Invertir el eje Y para que los mejores tiempos (menor número) queden arriba
    y=alt.Y('Marca_Num_Media', title='Media de Marca (Segundos) - Menor es Mejor', scale=alt.Scale(reverse=True))
).mark_line(point=True).properties(
    title='Tiempo (Carreras) - Mejora = Línea Ascendente'
)

# Combinar los gráficos
final_chart = (distancia_chart | tiempo_chart).configure_title(
    fontSize=18,
    anchor='middle'
).configure_axis(
    titleFontSize=14,
    labelFontSize=12
).configure_legend(
    titleFontSize=14,
    labelFontSize=12
)

# Guardar la visualización
# final_chart.save('Visualizacion_Tendencia_Anual_Metros.html')
print("Se generó la visualización de tendencia anual con Altair, ajustada para Distancia/Metros. Se puede guardar como HTML o JPG/PNG.")
# final_chart

Se generó la visualización de tendencia anual con Altair, ajustada para Distancia/Metros. Se puede guardar como HTML o JPG/PNG.
